In [42]:
import pandas as pd
from sklearn.cluster import SpectralCoclustering, SpectralBiclustering
import numpy as np

In [2]:
df = pd.read_parquet("../data/recipe_ingredient_matrix.parquet")

In [3]:
df.head()

,orange peel–infused vodka,lime juice,rhubarb syrup,Dimmi liqueur,Midori,blanco tequila,mezcal,"poblano liqueur, such as Ancho Reyes Verde","sparkling mineral water, such as Topo Chico","fresh pineapple, cubed",...,Latitude 29 Orgeat,Coruba Dark Rum,Denizen Merchant's Reserve 8-Year Rum,Orange Ice Cubes,Ice Cubes,Pre-Batched Cocktail,Orange Juice Blend,"prosecco, preferably a drier, sharp prosecco like Gambino Gold)",Batched Garibaldi Base,fresh squeezed orange juice
Rhubarbra Streisand,1.5,0.75,0.75,0.5,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Songs About Keri,0.0,0.50,0.00,0.0,0.75,0.5,0.5,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Nikita,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,150.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Green Gazoo,0.0,0.75,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Gradda’s Tamal,0.0,0.75,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Drop rows containing infs
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna(how='any')


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 997 entries, Rhubarbra Streisand to Fermented Citrus Garibaldi
Columns: 2646 entries, orange peel–infused vodka to fresh squeezed orange juice
dtypes: float64(2646)
memory usage: 20.2+ MB


In [36]:
clusters = SpectralCoclustering(n_clusters=20, svd_method="arpack")

In [40]:
# Remove any rows with infs or nans before converting to numpy array
df_clean = df.replace([np.inf, -np.inf], np.nan).dropna(how='any')
arr = df_clean.to_numpy()
# Convert to sparse array
from scipy.sparse import csr_matrix
arr = csr_matrix(arr)

In [41]:
clusters.fit(arr)

c:\Users\kurtt\miniforge3\envs\py313\Lib\site-packages\sklearn\cluster\_bicluster.py:32: RuntimeWarning: divide by zero encountered in divide
  col_diag = np.asarray(1.0 / np.sqrt(X.sum(axis=0))).squeeze()
c:\Users\kurtt\miniforge3\envs\py313\Lib\site-packages\sklearn\cluster\_bicluster.py:346: RuntimeWarning: invalid value encountered in multiply
  z = np.vstack((row_diag[:, np.newaxis] * u, col_diag[:, np.newaxis] * v))


ValueError: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [43]:
c = SpectralBiclustering(n_clusters=10)

In [44]:
c.fit(arr)

c:\Users\kurtt\miniforge3\envs\py313\Lib\site-packages\sklearn\cluster\_bicluster.py:32: RuntimeWarning: divide by zero encountered in divide
  col_diag = np.asarray(1.0 / np.sqrt(X.sum(axis=0))).squeeze()


,n_clusters,10
,method,'bistochastic'
,n_components,6
,n_best,3
,svd_method,'randomized'
,n_svd_vecs,None
,mini_batch,False
,init,'k-means++'
,n_init,10
,random_state,None


In [53]:
c.column_labels_[c.columns_[1]]

array([1], dtype=int32)

In [60]:
df.columns[c.column_labels_==4]

Index(['vodka'], dtype='object')

In [64]:
df.index[c.row_labels_==0]

Index(['Rhubarbra Streisand', 'Songs About Keri', 'Green Gazoo',
       'Gradda’s Tamal', 'Gin Coco No. 2', 'Beton', 'Coronation No. 1',
       'Santo Libre', 'Kangaroo', 'Strega Drop',
       ...
       'Fast Canoe', 'Leeward Negroni', 'Pandan-quiri', 'Rosé Piscine',
       'Butter Martini', 'Queen Bee', 'Garret Richard’s Mai Tai',
       'Groove is in the Heart', 'Garibaldi Spritz',
       'Fermented Citrus Garibaldi'],
      dtype='object', length=983)